## Aufgabe: 

Trainiere ein Modell, welches Katzen und Hunden unterscheiden kann!

### Lösungstipp 1:

Dieses Modell wird recht komplex werden, und entsprechend lange zum Training benötigen. Später werden wir noch ein schnelleres Modell kennenlernen - für diese Aufgabe möchtest du aber auf jeden Fall eine GPU zur Verfügung haben. 

### Lösungstipp 2:

Ggf. macht es Sinn, die Datenmenge erstmal zu reduzieren - beispielsweise könntest du mit 1000 Hund-Bildern und 1000 Katzen-Bildern starten. Du darfst den Code aus den letzten Lektionen 1:1 übernehmen, anpassen,...

### Lösungstipp 3:

Um die Bilder korrekt zu klassifizieren, wirst du u.U. mehrere Layer benötigen. Du könntest hier z.B. einen Aufbau ähnlich zum VGG-Netz verwenden, also mehrere Blöcke mit jeweils 2-3 Convolutional-Layern, die von einem MaxPooling abgeschlossen werden. Nach mehreren dieser Blöcke kannst du dann 2-3 Dense-Layer platzieren, um die Ergebnisse auf 1 Neuron herunterzubrechen.

### Lösungstipp 4:

Ggf. sind die y-Daten noch nicht im richtigen Format. Du brauchst hier eine Shape von (X, 1), wobei X hier die Anzahl der Datensätze sind. Die Shape der y-Variable kannst du dir über `y.shape` ausgeben, sollte die Shape nicht `(X, 1)` sein, so kannst du hierfür den `.reshape`-Befehl verwenden.

### Lösungstipp 5: 

Gegebenenfalls wird dein Netz trotzdem die Daten nicht korrekt lernen können. Das liegt dann daran, dass die Lernrate zu hoch ist - in dem Fall musst du die Lernrate verringern. Beispiel (für "adam"):

```python
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.000001), loss="binary_crossentropy", metrics=["acc"])
```


In [1]:
import os
import numpy as np
from tqdm import tqdm_notebook as tqdm
from PIL import Image

def read_images(path):
    files = os.listdir(path)
    files = [file for file in files if file[-4:] == ".jpg"]
    
    # files = files[:2000]
    
    images = []
    for file in tqdm(files):
        try:
            image = Image.open(os.path.join(path, file))
            image = image.resize((224, 224), Image.LANCZOS)
            image = image.convert("RGB")
            image = np.asarray(image)
            images.append(image)
        except OSError:
            pass
    
    return images
    
cats = read_images(os.path.join("data", "PetImages", "Cat"))
dogs = read_images(os.path.join("data", "PetImages", "Dog"))

dogs = np.asarray(dogs)
cats = np.asarray(cats)

X = np.concatenate([dogs, cats]).astype(np.float32) / 255.

y_dogs = np.zeros(len(dogs))
y_cats = np.ones(len(cats))

y = np.concatenate([y_dogs, y_cats])

<ipython-input-1-0a3fd85e7491>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm(files):


C:\Users\Shadow\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  warnings.warn(
C:\Users\Shadow\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  warnings.warn(
C:\Users\Shadow\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  warnings.warn(
C:\Users\Shadow\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  warnings.warn(
C:\Users\Shadow\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 283
  warnings.warn(
C:\Users\Shadow\anaconda3\lib\site-packages\PIL\T

In [2]:
# y ins richtige Format bringen
y = y.reshape(-1, 1)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(224, 224, 3)))
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D())

model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D())

model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D())

model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(1024, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(lr=0.00001), loss="binary_crossentropy", metrics=["acc"])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 220, 220, 32)      9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 108, 108, 64)      18496     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 106, 106, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 53, 53, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 51, 51, 128)      

In [5]:
model.fit(X, y, epochs=20, batch_size=64)

Epoch 1/20
391/391 [==============================] - 110s 244ms/step - loss: 0.6866 - acc: 0.5260
Epoch 2/20
391/391 [==============================] - 94s 240ms/step - loss: 0.6200 - acc: 0.6558
Epoch 3/20
391/391 [==============================] - 96s 246ms/step - loss: 0.5731 - acc: 0.6982
Epoch 4/20
391/391 [==============================] - 96s 245ms/step - loss: 0.5455 - acc: 0.7253
Epoch 5/20
391/391 [==============================] - 95s 244ms/step - loss: 0.5211 - acc: 0.7419
Epoch 6/20
391/391 [==============================] - 98s 251ms/step - loss: 0.5026 - acc: 0.7529
Epoch 7/20
391/391 [==============================] - 95s 242ms/step - loss: 0.4834 - acc: 0.7671
Epoch 8/20
391/391 [==============================] - 93s 239ms/step - loss: 0.4697 - acc: 0.7800
Epoch 9/20
391/391 [==============================] - 94s 239ms/step - loss: 0.4595 - acc: 0.7847
Epoch 10/20
391/391 [==============================] - 93s 239ms/step - loss: 0.4442 - acc: 0.7904
Epoch 11/20
391/39

In [6]:
model.save("dave.h5")